<a href="https://colab.research.google.com/github/claudelepere/ML_GitHub/blob/main/Longformer_for_multi_label_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q accelerate
!pip install -q huggingface_hub
!pip install -q scikit-learn
!pip install -q transformers datasets
!pip install -q wandb

import json
import numpy as np
import os
import sys
import time
import torch
import wandb

from datasets              import DatasetDict
from google.colab          import auth, drive, files, userdata
from huggingface_hub       import create_repo, login, upload_file
from huggingface_hub.utils import RepositoryNotFoundError
from sklearn.metrics       import accuracy_score, average_precision_score, classification_report, f1_score, precision_score, precision_recall_fscore_support, recall_score, roc_auc_score
from torch.utils.data      import DataLoader
from tqdm.auto             import tqdm
from transformers          import AdamW, EvalPrediction, LongformerTokenizerFast, LongformerForSequenceClassification, Trainer, TrainingArguments
from torch.nn              import BCEWithLogitsLoss


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
"""
# Check the Python version
print(sys.version)
print()

# Get the installed packages (you can see that conda is not installed (do not install it))
!pip list
print()

# Check system information
!cat /etc/os-release
!uname -m
print()

# Check the GPU details (only if the runtime type is T4 GPU)
#!nvidia-smi
#print()

# Check RAM
!free -h
print()

# Check disk space
!df -h
print()

# Get environment variables
for key, value in os.environ.items():
    print(f"{key}: {value}")
"""
!python -V

print(f"currentdir: {os.getcwd()}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

datasetDict_zip_file_name = "dataset_11_1000.zip"
datasetDict_dir_name      = os.path.splitext(datasetDict_zip_file_name)[0]
print(f"datasetDict_zip_file_name: {datasetDict_zip_file_name}")
print(f"datasetDict_dir_name     : {datasetDict_dir_name}")
print()

max_length    = 1024  # 4096 max
batch_size    = 1    #8
epochs        = 5
learning_rate = 2e-5
threshold     = 0.2

run_name = f"Longformer-multilabel-{datasetDict_dir_name}-batch{batch_size}-epochs{epochs}-lr{learning_rate}-threshold{threshold}"
print(f"run_name                 : {run_name}")

Python 3.10.12
currentdir: /content
device: cuda
datasetDict_zip_file_name: dataset_11_1000.zip
datasetDict_dir_name     : dataset_11_1000

run_name                 : Longformer-multilabel-dataset_11_1000-batch1-epochs5-lr2e-05-threshold0.2


In [3]:
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

!nvidia-smi
#!kill -9 <PID>

torch.cuda.empty_cache()

Fri Dec 20 18:01:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
def upload_unzip_dataset(file_name=datasetDict_zip_file_name):
  # Check if the file exists
  if not os.path.exists(file_name):
    print(f"'{file_name}' not found in /content. Uploading...")
    uploaded_files = files.upload()                              # Prompt file upload dialog
    if file_name not in uploaded_files:
      raise FileNotFoundError(f"'{file_name}' was not uploaded. Please try again.")
    print(f"'{file_name}' successfully uploaded to /content")
    uploaded_file_name = list(uploaded_files.keys())[0]          # Get the name of the uploaded file

    !unzip {uploaded_file_name}

    unzipped_dir_name = os.path.splitext(uploaded_file_name)[0]
    assert unzipped_dir_name==datasetDict_dir_name, "unzipped_dir_name != datasetDict_dir_name"
  else:
    print(f"'{datasetDict_dir_name}' already exists in /content.")


In [5]:
upload_unzip_dataset(datasetDict_zip_file_name)

'dataset_11_1000.zip' not found in /content. Uploading...


Saving dataset_11_1000.zip to dataset_11_1000.zip
'dataset_11_1000.zip' successfully uploaded to /content
Archive:  dataset_11_1000.zip
  inflating: dataset_11_1000/dataset_dict.json  
  inflating: dataset_11_1000/test/data-00000-of-00001.arrow  
  inflating: dataset_11_1000/test/dataset_info.json  
  inflating: dataset_11_1000/test/state.json  
  inflating: dataset_11_1000/train/data-00000-of-00001.arrow  
  inflating: dataset_11_1000/train/dataset_info.json  
  inflating: dataset_11_1000/train/state.json  
  inflating: dataset_11_1000/validation/data-00000-of-00001.arrow  
  inflating: dataset_11_1000/validation/dataset_info.json  
  inflating: dataset_11_1000/validation/state.json  


In [6]:
# Hugging Face Authenticate

os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")    # Store the key in os.environ
hf_token               = os.environ.get('HF_TOKEN')
login(token=hf_token)

# Verify
!huggingface-cli whoami

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


claudelepere


In [7]:
# Create the skill_classification repo on the Hugging Face Hub

HF_name         = "claudelepere/skill_classification"
repo_id_model   = HF_name
repo_id_dataset = HF_name

repo_model_url = create_repo(
    repo_id   = repo_id_model,
    repo_type = "model",
    private   = True,
    exist_ok  = True
)
print(f"Repo model url: {repo_model_url} created successfully as a private repo.")

repo_dataset_url = create_repo(
    repo_id   = repo_id_dataset,
    repo_type = "dataset",
    private   = True,
    exist_ok  = True
)
print(f"Repo datasets url: {repo_dataset_url} created successfully as a private repo.")

repo_id_dataset = f"datasets/{HF_name}"

print(f"repo_id_model: {repo_id_model}")
print(f"repo_id_dataset: {repo_id_dataset}")

Repo model url: https://huggingface.co/claudelepere/skill_classification created successfully as a private repo.
Repo datasets url: https://huggingface.co/datasets/claudelepere/skill_classification created successfully as a private repo.
repo_id_model: claudelepere/skill_classification
repo_id_dataset: datasets/claudelepere/skill_classification


In [8]:
# W&B initialization

os.environ["WANDB_API_KEY"] = userdata.get("WANDB_API_KEY")        # Store the key in os.environ
wandb_api_key               = os.environ.get('WANDB_API_KEY')
wandb.login(key=wandb_api_key)

try:
  wandb.init(
      project = "skill_classification",
      name    = run_name,
      entity  = "claudelepere-c-cile-cy",
      config  = {
          "learning_rate": learning_rate,
          "epochs"       : 5,
          "batch_size"   : batch_size
      }
  )
except wandb.errors.CommError as err:
  print(f"CommError: {err}")
except Exception as exc:
  print(f"Exception: {exc}")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: claudelepere (claudelepere-c-cile-cy). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [9]:
# Create the dataset: 3 Hugging Face Dataset in a Hugging Face DatasetDict

datasetDict = DatasetDict.load_from_disk(datasetDict_dir_name)

print(f"datasetDict: {type(datasetDict)} {datasetDict.shape}\n{datasetDict}")


datasetDict: <class 'datasets.dataset_dict.DatasetDict'> {'train': (700, 8), 'validation': (150, 8), 'test': (150, 8)}
DatasetDict({
    train: Dataset({
        features: ['id', 'text', '390', '135', '136', '137', '138', '139'],
        num_rows: 700
    })
    validation: Dataset({
        features: ['id', 'text', '390', '135', '136', '137', '138', '139'],
        num_rows: 150
    })
    test: Dataset({
        features: ['id', 'text', '390', '135', '136', '137', '138', '139'],
        num_rows: 150
    })
})


In [10]:
example = datasetDict['train'][0]
print(f"example: {type(example)} {example.keys()}\n{example}")

example: <class 'dict'> dict_keys(['id', 'text', '390', '135', '136', '137', '138', '139'])
{'id': 307149, 'text': "Umicore - Senior Network Platform engineer BGP,OSPF,VLAN, routers, switches Umicore About Umicore Reducing harmful vehicle emissions. Giving new life to used metals. Powering the cars of the future. As a global materials and technology group, we apply our specialist knowledge to offer materials and solutions that are essential to everyday life. We aim to be a clear world leader in materials for clean mobility and recycling and have turned our sustainability approach into an even greater competitive advantage. With ambitions like this, imagine what you could do? About our support services (Information Systems) Umicore's commitment to creating a sustainable future relies on innovation in all areas of business. As we keep pushing our boundaries and strive to set new industry standards, embracing advanced technologies has never been more essential. Digital technologies are co

In [11]:
# Create the label list and the id2label and label2id mappings.

"""
dataset 7_1000_125_125  ,  48 labels
dataset 7_128_18_54     ,  42 labels
dataset 8910_1087_68_204, 206 labels
dataset 11_1000         ,   6 labels
"""

labels = [label for label in datasetDict['train'].features.keys() if label not in ['id', 'text']]
labels.sort()
print(f"labels: {type(labels)} {len(labels)}\n{labels}")

id2label = {idx:label for idx, label in enumerate(labels)}
print(f"id2label: {type(id2label)} {len(id2label)}\n{id2label}")

label2id = {label:idx for idx, label in enumerate(labels)}
print(f"label2id: {type(label2id)} {len(label2id)}\n{label2id}")

labels: <class 'list'> 6
['135', '136', '137', '138', '139', '390']
id2label: <class 'dict'> 6
{0: '135', 1: '136', 2: '137', 3: '138', 4: '139', 5: '390'}
label2id: <class 'dict'> 6
{'135': 0, '136': 1, '137': 2, '138': 3, '139': 4, '390': 5}


In [12]:
# Load tokenizer and model

tokenizer = LongformerTokenizerFast.from_pretrained("allenai/longformer-base-4096")

model = LongformerForSequenceClassification.from_pretrained(
    "allenai/longformer-base-4096",
    num_labels=len(labels),
    problem_type='multi_label_classification')

# Configure attention window size
model.config.attention_window = 512

optimizer = AdamW(model.parameters(), lr=learning_rate)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/597M [00:00<?, ?B/s]

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [13]:
# Tokenize ('input_ids' and 'attention_mask'), add 'global_attention_mask' (for Longformer), add 'labels'

def preprocess_data(examples, indices):
  # Step 1: Extract text and tokenize
  text = examples['text']             # Batch of texts
  encoding = tokenizer(
      text,                           # Tokenize text
      truncation     = True,
      padding        = 'max_length',
      max_length     = max_length,
      return_tensors = 'pt'           # Return PyTorch tensors
  )

  # Step 2: Create and add the global attention mask
  global_attention_mask             = torch.zeros_like(encoding['input_ids'])  # Initialize global attention mask with zeros (same shape as input_ids)
  global_attention_mask[:, 0]       = 1                                        # Set global attention on the first token ([CLS], token ID=0) in each sequence
  encoding['global_attention_mask'] = global_attention_mask                    # Add the global_attention_mask to the batch

  # Step 3: Create and populate the label matrix

  labels_matrix = torch.zeros((len(text), len(labels)), dtype=torch.float32)   # Create an empty label matrix
  #print(f"labels_matrix: {type(labels_matrix)} {labels_matrix.shape}")

  # Populate label matrix
  for idx, label in enumerate(labels):
    #print(f"idx:{idx} label:{label}")
    if label in examples:
      labels_matrix[:, idx] = torch.tensor(
          [1.0 if val else 0.0 for val in examples[label]],
          dtype=torch.float32
          )
  #print(f"labels_matrix: {type(labels_matrix)} {labels_matrix.shape}")

  encoding['labels'] = labels_matrix                                           # Add labels to the encoding
  #print(f"encoding['labels']: {encoding['labels']}")

  return encoding

In [14]:
# Create the 3 encoded datasets, train, validation and test

encoded_dataset = datasetDict.map(
    preprocess_data,
    batched        = True,
    remove_columns = datasetDict['train'].column_names,
    with_indices   = True
    )
train_dataset      = encoded_dataset['train']
validation_dataset = encoded_dataset['validation']
test_dataset       = encoded_dataset['test']
print(f"encoded_dataset: {type(encoded_dataset)} {encoded_dataset.shape}\n{encoded_dataset}")
print(f"train_dataset: {type(train_dataset)} {train_dataset.shape}")
print(f"validation_dataset: {type(validation_dataset)} {validation_dataset.shape}")
print(f"test_dataset['labels']: {type(test_dataset['labels'])} {len(test_dataset['labels'])}\n{test_dataset['labels']}")

print(f"train_dataset[0]['input_ids']: {type(train_dataset[0]['input_ids'])} {len(train_dataset[0]['input_ids'])}\n{train_dataset['input_ids'][0]}")
print(f"train_dataset[0]['attention_mask']: {type(train_dataset[0]['attention_mask'])} {len(train_dataset[0]['attention_mask'])}\n{train_dataset['attention_mask'][0]}")
print(f"train_dataset[0]['global_attention_mask']: {type(train_dataset[0]['global_attention_mask'])} {len(train_dataset[0]['global_attention_mask'])}\n{train_dataset['global_attention_mask'][0]}")

print(f"train_dataset[0]['labels']: {type(train_dataset[0]['labels'])} {len(train_dataset[0]['labels'])}\n{train_dataset[0]['labels']}")
print(f"train_dataset['labels'][0]: {type(train_dataset['labels'][0])} {len(train_dataset['labels'][0])}\n{train_dataset['labels'][0]}")


Map:   0%|          | 0/700 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

encoded_dataset: <class 'datasets.dataset_dict.DatasetDict'> {'train': (700, 4), 'validation': (150, 4), 'test': (150, 4)}
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'global_attention_mask', 'labels'],
        num_rows: 700
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'global_attention_mask', 'labels'],
        num_rows: 150
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'global_attention_mask', 'labels'],
        num_rows: 150
    })
})
train_dataset: <class 'datasets.arrow_dataset.Dataset'> (700, 4)
validation_dataset: <class 'datasets.arrow_dataset.Dataset'> (150, 4)
test_dataset['labels']: <class 'list'> 150
[[0.0, 0.0, 1.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0, 1.0, 0.0], [0.0, 0.0, 1.0, 1.0, 1.0, 0.0], [0.0, 0.0, 1.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0, 1.0, 0.0], [0.0, 0.0, 1.0, 1.0, 1.0, 0.0], [0.0, 0.0, 0.

In [15]:
# Truncated part

def get_truncated_part(text):
  tokens = tokenizer(
      text,
      truncation                = True,
      padding                   = 'max_length',
      max_length                = 4096,
      return_overflowing_tokens = True,
      return_tensors            = None
      )
  print(f"tokens.keys(): {tokens.keys()}")

  # Get the truncated tokens
  truncated_ids = tokens["input_ids"][0]
  print(f"truncated_ids: {type(truncated_ids)} {truncated_ids}")
  #overflow_ids  = tokens["overflow_to_sample_mapping"][0]
  #print(f"overflow_ids: {type(overflow_ids)} {overflow_ids}")

  # Decode the tokens back to text
  truncated_text = tokenizer.decode(truncated_ids, skip_special_tokens=True)
  #overflow_text  = tokenizer.decode(overflow_ids, skip_special_tokens=True)

  print(f"original_text :\n{text}")
  print(f"truncated_text:\n{truncated_text}")
  #print(f"overflow_text:\n{overflow_text}")

  original_tokens  = tokenizer.tokenize(text)
  truncated_tokens = tokenizer.tokenize(truncated_text)
  #overflow_tokens  = tokenizer.tokenize(overflow_text)

  print(f"original_tokens count : {len(original_tokens)}")
  print(f"truncated_tokens count: {len(truncated_tokens)}")
  #print(f"overflow_tokens count: {len(overflow_tokens)}")


In [16]:
example_text = datasetDict['train'][0]['text']
#get_truncated_part(example_text)

inputs = tokenizer(
    example_text,
    truncation     = True,
    padding        = 'max_length',
    max_length     = max_length,
    return_tensors = 'pt'
)

print(f"inputs: {type(inputs)} {inputs.keys()}\n{inputs}")
print(f"inputs_ids: {type(inputs.input_ids)} {inputs.input_ids.shape}\n{inputs.input_ids}")
print(f"attention_mask: {type(inputs.attention_mask)} {inputs.attention_mask.shape}\n{inputs.attention_mask}")
#print(f"token_type_ids: {inputs.token_type_ids.shape}")
#print(f"labels: {inputs.labels.shape}")


inputs: <class 'transformers.tokenization_utils_base.BatchEncoding'> dict_keys(['input_ids', 'attention_mask'])
{'input_ids': tensor([[    0,   791, 15796,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])}
inputs_ids: <class 'torch.Tensor'> torch.Size([1, 1024])
tensor([[    0,   791, 15796,  ...,     1,     1,     1]])
attention_mask: <class 'torch.Tensor'> torch.Size([1, 1024])
tensor([[1, 1, 1,  ..., 0, 0, 0]])


In [17]:
# 4. Forward pass for multi-label classification

outputs = model(
    input_ids      = inputs.input_ids,
    attention_mask = inputs.attention_mask
    )

print(f"outputs: {type(outputs)} {outputs.keys()}\n{outputs}")

# Logits (= raw model outputs)
logits = outputs.logits
print(f"logits: {type(logits)} {logits.shape}\n{logits}")

# Convert logits to probabilities
sigmoid = torch.nn.Sigmoid()
probs   = sigmoid(logits)
print(f"probs: {type(probs)} {probs.shape}\n{probs}")

Initializing global attention on CLS token...


outputs: <class 'transformers.models.longformer.modeling_longformer.LongformerSequenceClassifierOutput'> odict_keys(['logits'])
LongformerSequenceClassifierOutput(loss=None, logits=tensor([[ 0.1644,  0.0731, -0.0697,  0.0809,  0.0559,  0.1036]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None, global_attentions=None)
logits: <class 'torch.Tensor'> torch.Size([1, 6])
tensor([[ 0.1644,  0.0731, -0.0697,  0.0809,  0.0559,  0.1036]],
       grad_fn=<AddmmBackward0>)
probs: <class 'torch.Tensor'> torch.Size([1, 6])
tensor([[0.5410, 0.5183, 0.4826, 0.5202, 0.5140, 0.5259]],
       grad_fn=<SigmoidBackward0>)


In [18]:
example = encoded_dataset['train'][0]

print(f"example: {type(example)} {example.keys()}\n{example}")
print()
print(f"example['input_ids']: {type(example['input_ids'])} {len(example['input_ids'])}\n{example['input_ids']}")
#print(f"example['token_type_ids']: {type(example['token_type_ids'])} {len(example['token_type_ids'])}\n{example['token_type_ids']}")
print(f"example['attention_mask']: {type(example['attention_mask'])} {len(example['attention_mask'])}\n{example['attention_mask']}")
print(f"example['labels']:  {type(example['labels'])} {len(example['labels'])}\n{example['labels']}")

example: <class 'dict'> dict_keys(['input_ids', 'attention_mask', 'global_attention_mask', 'labels'])
{'input_ids': [0, 791, 15796, 1688, 111, 3596, 3658, 16237, 8083, 163, 12694, 6, 3196, 16088, 6, 846, 20606, 6, 37579, 6, 21737, 121, 15796, 1688, 1936, 121, 15796, 1688, 1211, 30623, 11190, 1155, 5035, 4, 19506, 92, 301, 7, 341, 10340, 4, 3029, 154, 5, 1677, 9, 5, 499, 4, 287, 10, 720, 3183, 8, 806, 333, 6, 52, 3253, 84, 6857, 2655, 7, 904, 3183, 8, 2643, 14, 32, 4499, 7, 7476, 301, 4, 166, 4374, 7, 28, 10, 699, 232, 884, 11, 3183, 13, 2382, 11525, 8, 11690, 8, 33, 1224, 84, 11128, 1548, 88, 41, 190, 2388, 2695, 2093, 4, 590, 12294, 101, 42, 6, 4744, 99, 47, 115, 109, 116, 1936, 84, 323, 518, 36, 38741, 5778, 43, 121, 15796, 1688, 18, 2720, 7, 2351, 10, 5068, 499, 12438, 15, 4695, 11, 70, 911, 9, 265, 4, 287, 52, 489, 3784, 84, 10156, 8, 15407, 7, 278, 92, 539, 2820, 6, 16105, 3319, 4233, 34, 393, 57, 55, 4499, 4, 6282, 4233, 32, 5861, 14007, 8, 1455, 92, 1616, 13, 201, 4, 287, 220, 2

In [19]:
tokenizer.decode(example['input_ids'])

"<s>Umicore - Senior Network Platform engineer BGP,OSPF,VLAN, routers, switches Umicore About Umicore Reducing harmful vehicle emissions. Giving new life to used metals. Powering the cars of the future. As a global materials and technology group, we apply our specialist knowledge to offer materials and solutions that are essential to everyday life. We aim to be a clear world leader in materials for clean mobility and recycling and have turned our sustainability approach into an even greater competitive advantage. With ambitions like this, imagine what you could do? About our support services (Information Systems) Umicore's commitment to creating a sustainable future relies on innovation in all areas of business. As we keep pushing our boundaries and strive to set new industry standards, embracing advanced technologies has never been more essential. Digital technologies are constantly evolving and present new opportunities for us. As next generation corporate IT department, we design & 

In [20]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['138', '139']

In [21]:
# Set PyTorch format to ensures correctness and compatibility with PyTorch pipelines

# The 3 Hugging Face Dataset are formatted as PyTorch Dataset
encoded_dataset.set_format('torch')

In [22]:
batch_size  = batch_size
metric_name = "f1"

In [23]:
args = TrainingArguments(
    output_dir                  = './training_results',  # where model predictions and checkpoints will be written during training
    overwrite_output_dir        = True,
    logging_dir                 = './logs',
    logging_steps               = 50,
    save_steps                  = 500,
    save_total_limit            = 2,
    eval_strategy               = 'epoch',
    save_strategy               = 'epoch',
    learning_rate               = learning_rate,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size  = batch_size,
    num_train_epochs            = epochs,
    weight_decay                = 0.01,
    load_best_model_at_end      = True,
    metric_for_best_model       = metric_name,
    run_name                    = run_name,
    gradient_accumulation_steps = 4,
    fp16                        = True
    #report_to                  = 'wandb'
    )

In [24]:
# Metrics
#   source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/

def multi_label_metrics(predictions, labels):
    average = 'micro'    # 'micro' or 'weighted'

    # first, apply sigmoid on predictions whose shape is (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs   = sigmoid(torch.Tensor(predictions))

    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1

    # finally, compute metrics
    y_true               = labels
    f1                   = f1_score               (y_true=y_true, y_pred=y_pred, average=average)    #, zero_division=1)
    precision            = precision_score        (y_true=y_true, y_pred=y_pred, average=average)    #, zero_division=1)
    recall               = recall_score           (y_true=y_true, y_pred=y_pred, average=average)    #, zero_division=1)
    roc_auc              = roc_auc_score          (y_true=y_true, y_score=probs, average=average)
    precision_recall_auc = average_precision_score(y_true=y_true, y_score=probs, average=average)
    accuracy             = accuracy_score         (y_true=y_true, y_pred=y_pred)

    # return as dictionary
    metrics = {
        'f1'                  : f1,
        'precision'           : precision,
        'recall'              : recall,
        'roc_auc'             : roc_auc,
        'precision_recall_auc': precision_recall_auc,
        'accuracy'            : accuracy
        }

    return metrics

In [25]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(
        predictions = preds,
        labels      = p.label_ids
        )
    return result

Let's verify a batch as well as a forward pass:

In [26]:
print(f"inputids:        {type(encoded_dataset['train']['input_ids'][0])}\t{encoded_dataset['train']['input_ids'][0].shape}")
#print(f"token_type_ids': {type(encoded_dataset['train']['token_type_ids'][0])}\t{encoded_dataset['train']['token_type_ids'][0].shape}")
print(f"attention_mask:  {type(encoded_dataset['train']['attention_mask'][0])}\t{encoded_dataset['train']['attention_mask'][0].shape}")
print(f"labels:          {type(encoded_dataset['train'][0]['labels'])}\t{encoded_dataset['train'][0]['labels'].shape}")

inputids:        <class 'torch.Tensor'>	torch.Size([1024])
attention_mask:  <class 'torch.Tensor'>	torch.Size([1024])
labels:          <class 'torch.Tensor'>	torch.Size([6])


In [27]:
# Execute a forward pass for debugging or verificatin purposes (cf. BERT_3_1 in Notion BERT database)

#outputs = model(
#    input_ids      = encoded_dataset['train']['input_ids'][0].unsqueeze(0),
#    attention_mask = encoded_dataset['train']['attention_mask'][0].unsqueeze(0),
#    labels         = encoded_dataset['train'][0]['labels'].unsqueeze(0)
#    )

#print(f"outputs: {type(outputs)} {outputs.keys()}\n{outputs}")

In [28]:
# Create the trainer

trainer = Trainer(
    model,
    args,
    train_dataset = encoded_dataset["train"],
    eval_dataset  = encoded_dataset["validation"],
    tokenizer     = tokenizer,
    compute_metrics=compute_metrics
    )


<ipython-input-28-2d6589e12c30>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [29]:
# Train, save the results as a JSON file

train_output  = trainer.train()

train_results = {
    'global_step':   train_output.global_step,    # total steps completed during training
    'training_loss': train_output.training_loss,  # average loss during training
    'metrics':       train_output.metrics         # dictionary of metrics
}

# Save train results
with open("train_results.json", "w") as f:
  json.dump(train_results, f, indent=4)
print(f"train_results: {type(train_results)} {len(train_results)}\n{train_results}")

Epoch,Training Loss,Validation Loss,F1,Precision,Recall,Roc Auc,Precision Recall Auc,Accuracy
1,0.360900,0.415405,0.784713,0.684444,0.919403,0.890440,0.801061,0.233333
2,0.346800,0.410930,0.780303,0.676149,0.922388,0.901284,0.812487,0.213333
3,0.341900,0.415978,0.789333,0.713253,0.883582,0.901178,0.794522,0.366667
4,0.258600,0.397618,0.790757,0.693694,0.919403,0.906792,0.813877,0.293333
5,0.212300,0.399002,0.793103,0.713604,0.892537,0.908247,0.813547,0.360000


train_results: <class 'dict'> 3
{'global_step': 875, 'training_loss': 0.29929059110369, 'metrics': {'train_runtime': 1205.9777, 'train_samples_per_second': 2.902, 'train_steps_per_second': 0.726, 'total_flos': 2299046787072000.0, 'train_loss': 0.29929059110369, 'epoch': 5.0}}


In [30]:
print("Training successfully completed.")

Training successfully completed.


## Evaluate

After training, we evaluate our model on the validation set.

In [31]:
def get_results(model, dataset, batch_size, threshold):

  # Set the model to evaluation mode to disable dropout and other training-specific behaviors
  model.eval()

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)

  test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

  all_preds       = []
  all_probs       = []
  all_true_labels = []

  for batch in tqdm(test_loader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
      outputs = model(**batch)
    logits = outputs.logits

    # Convert logits to probabilities and probabilities to predictions
    sigmoid = torch.nn.Sigmoid()
    probs   = sigmoid(logits).cpu().numpy()    # Convert to Numpy
    preds   = (probs > threshold).astype(int)  # Convert to binary Numpy array

    # Accumulate probabilities, predictions and labels
    all_probs.append(probs)
    all_preds.append(preds)
    all_true_labels.append(batch['labels'].cpu().numpy())

  # Concatenate results from all batches
  all_probs       = np.concatenate(all_probs, axis=0)        # shape: [num_samples, num_labels]
  all_preds       = np.concatenate(all_preds, axis=0)        # shape: [num_samples, num_labels]
  all_true_labels = np.concatenate(all_true_labels, axis=0)  # shape: [num_samples, num_labels]

  print(f"all_probs:       {type(all_probs)} {all_probs.shape}")
  print(f"all_preds:       {type(all_preds)} {all_preds.shape}")
  print(f"all_true_labels: {type(all_true_labels)} {all_true_labels.shape}")

  # Classification report for precision, recall, F1 score
  print(classification_report(
      y_true        = all_true_labels,
      y_pred        = all_preds,
      target_names  = labels,
      zero_division = 0
      ))

  # ROC AUC for multi-label classification
  roc_auc = roc_auc_score(
      y_true  = all_true_labels,
      y_score = all_probs,
      average = 'micro'
      )
  print(f"ROC AUC: {roc_auc}")

In [32]:
# First evaluate results NO SAVE

get_results(model=model, dataset=validation_dataset, batch_size=batch_size, threshold=threshold)

  0%|          | 0/150 [00:00<?, ?it/s]

all_probs:       <class 'numpy.ndarray'> (150, 6)
all_preds:       <class 'numpy.ndarray'> (150, 6)
all_true_labels: <class 'numpy.ndarray'> (150, 6)
              precision    recall  f1-score   support

         135       1.00      0.20      0.33         5
         136       0.20      0.25      0.22        16
         137       0.65      0.86      0.74        81
         138       0.85      1.00      0.92       128
         139       0.69      0.97      0.80        99
         390       0.00      0.00      0.00         6

   micro avg       0.71      0.89      0.79       335
   macro avg       0.57      0.55      0.50       335
weighted avg       0.71      0.89      0.78       335
 samples avg       0.73      0.91      0.79       335

ROC AUC: 0.9082472592788271


In [33]:
print("First evaluation successfully completed.")

First evaluation successfully completed.


In [34]:
# Second evaluate results; save to /content

eval_output = trainer.evaluate()

# Save evaluate results
with open("eval_results.json", "w") as f:
  json.dump(eval_output, f, indent=4)

In [35]:
print("Second evaluation successfully completed.")

Second evaluation successfully completed.


## Upload model, tokenizer, train results, evaluate results

In [36]:
# Save model to /content

model_path = "model"
trainer.save_model(model_path)

In [37]:
# Upload model and tokenizer to the HF repo_id_model

tokenizer = LongformerTokenizerFast.from_pretrained(model_path)
model     = LongformerForSequenceClassification.from_pretrained(model_path)

tokenizer.push_to_hub(repo_id_model)
model.push_to_hub(repo_id_model)


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/595M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/claudelepere/skill_classification/commit/436a3fb6fa690adeefbd0251bcc6c6c765df9731', commit_message='Upload LongformerForSequenceClassification', commit_description='', oid='436a3fb6fa690adeefbd0251bcc6c6c765df9731', pr_url=None, repo_url=RepoUrl('https://huggingface.co/claudelepere/skill_classification', endpoint='https://huggingface.co', repo_type='model', repo_id='claudelepere/skill_classification'), pr_revision=None, pr_num=None)

In [38]:
# Upload train_results.json and eval_results.json to the HF repo_id_dataset BETTER to upload to wanddb repo?

upload_file(
    path_or_fileobj = "train_results.json",
    path_in_repo    = "train_results.json",
    repo_id         = HF_name,
    repo_type       = "dataset"
    )

upload_file(
    path_or_fileobj = "eval_results.json",
    path_in_repo    = "eval_results.json",
    repo_id         = HF_name,
    repo_type       = "dataset"
    )

ValueError: Provided path: 'train_results.json' is not a file on the local file system

## Test

In [39]:
# Test: first results NO SAVE

get_results(model=model, dataset=test_dataset, batch_size=batch_size, threshold=threshold)

  0%|          | 0/150 [00:00<?, ?it/s]

all_probs:       <class 'numpy.ndarray'> (150, 6)
all_preds:       <class 'numpy.ndarray'> (150, 6)
all_true_labels: <class 'numpy.ndarray'> (150, 6)
              precision    recall  f1-score   support

         135       1.00      0.20      0.33         5
         136       0.22      0.33      0.27        12
         137       0.72      0.94      0.82        84
         138       0.87      1.00      0.93       131
         139       0.69      0.99      0.81       101
         390       0.00      0.00      0.00         8

   micro avg       0.74      0.92      0.82       341
   macro avg       0.58      0.58      0.53       341
weighted avg       0.74      0.92      0.81       341
 samples avg       0.75      0.94      0.82       341

ROC AUC: 0.9156852150100464


In [40]:
print("First test successfully completed.")

First test successfully completed.


In [41]:
# Test: second results NO SAVE

predictions = trainer.predict(test_dataset)

#print(f"predictions.predictions: {type(predictions.predictions)} {predictions.predictions.shape}\n{predictions.predictions}")  # Model logits
#print(f"predictions.label_ids: {type(predictions.label_ids)} {predictions.label_ids.shape}\n{predictions.label_ids}")          # Ground truth labels
print(f"predictions.metrics: {type(predictions.metrics)} {len(predictions.metrics)}\n{predictions.metrics}")                  # Metrics


predictions.metrics: <class 'dict'> 10
{'test_loss': 0.3683197498321533, 'test_f1': 0.824607329842932, 'test_precision': 0.7446808510638298, 'test_recall': 0.9237536656891495, 'test_roc_auc': 0.9157193144439955, 'test_precision_recall_auc': 0.8699920110662543, 'test_accuracy': 0.36, 'test_runtime': 12.9348, 'test_samples_per_second': 11.597, 'test_steps_per_second': 11.597}


In [42]:
print("Second test successfully completed.")

Second test successfully completed.


### Or otherwise

In [43]:
# Test: third results NO SAVE

predictions = trainer.predict(test_dataset)
print(predictions.predictions)  # Model logits
print(predictions.label_ids)    # Ground truth labels
print(predictions.metrics)      # Metrics

[[-4.2226562  -2.5800781   2.2871094   3.1679688   0.5878906  -3.9277344 ]
 [-3.7441406  -3.5878906  -1.9550781   2.1875      2.9707031  -3.6386719 ]
 [-3.7421875  -3.5742188  -1.8515625   2.2988281   3.0664062  -3.4960938 ]
 [-3.5839844  -1.2949219   3.2011719   3.0039062   1.3623047  -3.2246094 ]
 [-3.6875     -1.3515625   3.1113281   3.0527344   1.4619141  -3.2871094 ]
 [-4.234375   -3.609375   -0.7089844   2.8945312   2.8925781  -3.9160156 ]
 [-3.3183594  -3.4257812  -2.4042969   1.8095703   3.3203125  -3.0546875 ]
 [-4.0195312  -3.5800781  -1.1884766   2.6289062   3.3339844  -3.71875   ]
 [-3.7421875  -3.5976562  -2.0683594   2.0917969   2.8339844  -3.5410156 ]
 [-4.2578125  -3.0546875   1.3710938   3.3222656   2.3164062  -3.8613281 ]
 [-2.2558594  -1.0625      2.6230469   1.4980469  -2.1425781  -2.3691406 ]
 [-3.3203125  -1.015625    3.3242188   2.8613281   1.3007812  -3.0019531 ]
 [-4.0742188  -3.53125    -0.48046875  2.7558594   2.0507812  -4.0390625 ]
 [-3.7109375  -3.5585938 

In [44]:
print("Third test successfully completed.")

Third test successfully completed.
